In [ ]:
from pathlib import Path
import re, os
import pandas as pd

In [ ]:
dir = "D:/Datasets/synthetic/signal"

In [ ]:
def check_data(dir, folder='rx_x'):
    paths = []
    sizes = []
    for _, _, files in os.walk(os.path.join(dir, folder)):
        for file in files:
            if file.endswith(".mat"):
                paths.append(file)
                sizes.append(os.path.getsize(os.path.join(dir, folder, file)))
    df_path = pd.DataFrame({'path': paths, 'size': sizes})
    df_path = pd.concat([
        df_path, df_path['path'].str.extract(
            r'(?P<channel>[\w]+)_(?P<modulation>[\w-]+)_(?P<fs>[\w.+-]+)_(?P<sps>[\w.+-]+)_(?P<rolloff>[\w.+-]+)_(?P<snr>[\w.+-]+)\.mat')], axis=1)
    df_path['fs'] = df_path['fs'].astype(float)
    df_path['sps'] = df_path['sps'].astype(float)
    df_path['rolloff'] = df_path['rolloff'].astype(float)
    df_path['snr'] = df_path['snr'].astype(float)
    return df_path.groupby(['channel', 'fs', 'sps', 'rolloff'])['size'].agg(['min', 'max', 'count'])

In [ ]:
check_data(dir, 'rx_x')

In [ ]:
check_data(dir, 'y')

In [ ]:
check_data(dir, 'rx_s')

In [ ]:
check_data(dir, 'rx_nf')

In [ ]:
check_data(dir, 'tx_s')

In [ ]:
def delete(dir):
    df = check_data(dir, 'rx_x')
    df = df.loc[(df['min'] < 3e7) | (df['count'] < 260)]

    paths = []
    sizes = []
    for _, _, files in os.walk(os.path.join(dir, 'rx_x')):
        for file in files:
            if file.endswith(".mat"):
                paths.append(file)
                sizes.append(os.path.getsize(os.path.join(dir, 'rx_x', file)))
    df_path = pd.DataFrame({'path': paths, 'size': sizes})
    df_path = pd.concat([
        df_path, df_path['path'].str.extract(
            r'(?P<channel>[\w]+)_(?P<modulation>[\w-]+)_(?P<fs>[\w.+-]+)_(?P<sps>[\w.+-]+)_(?P<rolloff>[\w.+-]+)_(?P<snr>[\w.+-]+)\.mat')], axis=1)
    df_path['fs'] = df_path['fs'].astype(float)
    df_path['sps'] = df_path['sps'].astype(float)
    df_path['rolloff'] = df_path['rolloff'].astype(float)
    df_path['snr'] = df_path['snr'].astype(float)

    df_delete = df_path.merge(df, how='inner', on=['channel', 'fs', 'sps', 'rolloff'])
    for file in df_delete.path:
        path = os.path.join(dir, 'rx_x', file)
        if os.path.exists(path):
            os.remove(path)
    paths = []
    for _, _, files in os.walk(os.path.join(dir, 'rx_x')):
        for file in files:
            if file.endswith(".mat"):
                paths.append(file)
    for g in ['rx_s', 'rx_nf', 'y', 'tx_s']:
        for _, _, files in os.walk(os.path.join(dir, g)):
            for file in files:
                if file not in paths:
                    path = os.path.join(dir, g, file)
                    os.remove(path)

delete(dir)